In [ ]:
#Evaluación de Xception variando parámetros individuales, incluyendo tolerancia

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import os
import pandas as pd
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Ruta y carga de datos
ruta_base = '/content/drive/MyDrive/SCUBI DU'
def is_valid_image(file_path):
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    return any(file_path.lower().endswith(ext) for ext in valid_extensions)

clases_unicas = sorted([d for d in os.listdir(ruta_base) if os.path.isdir(os.path.join(ruta_base, d))])
class_to_label = {name: idx for idx, name in enumerate(clases_unicas)}

imagenes_cargadas, etiquetas_cargadas = [], []
for class_name in clases_unicas:
    class_folder = os.path.join(ruta_base, class_name)
    valid_images = [os.path.join(class_folder, f) for f in os.listdir(class_folder) if is_valid_image(f)]
    for image_path in valid_images:
        img = load_img(image_path, target_size=(224, 224))
        imagenes_cargadas.append(np.array(img))
        etiquetas_cargadas.append(class_to_label[class_name])

X = np.array(imagenes_cargadas) / 255.0
y = to_categorical(np.array(etiquetas_cargadas))

# Dividir en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

# Valores por defecto y por testear
parametros = {
    'batch_size': {'valores': [2, 6, 12], 'por_defecto': 6},
    'activation': {'valores': ['relu', 'tanh', 'sigmoid'], 'por_defecto': 'relu'},
    'patience': {'valores': [3, 5, 10], 'por_defecto': 5},
    'epochs': {'valores': [5, 10, 20], 'por_defecto': 10},
    'dropout_layers': {'valores': [0, 1, 2], 'por_defecto': 1},
    'learning_rate': {'valores': [0.0001, 0.001, 0.01], 'por_defecto': 0.001},
    'tolerancia': {'valores': [0.1, 0.001, 0], 'por_defecto': 0}
}

# Crear modelo base con capas clasificadoras
def build_model(activation='relu', dropout_layers=1, learning_rate=0.001):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())

    for _ in range(dropout_layers):
        model.add(layers.Dense(256, activation=activation))
        model.add(layers.Dropout(0.5))

    model.add(layers.Dense(y.shape[1], activation='softmax'))
    opt = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Almacenar resultados
resultados = []

# Bucle para cada parámetro
for param, config in parametros.items():
    for val in config['valores']:
        # Setear parámetros actuales y por defecto
        p = {
            'batch_size': parametros['batch_size']['por_defecto'],
            'activation': parametros['activation']['por_defecto'],
            'patience': parametros['patience']['por_defecto'],
            'epochs': parametros['epochs']['por_defecto'],
            'dropout_layers': parametros['dropout_layers']['por_defecto'],
            'learning_rate': parametros['learning_rate']['por_defecto'],
            'tolerancia': parametros['tolerancia']['por_defecto']
        }
        p[param] = val

        model = build_model(activation=p['activation'], dropout_layers=p['dropout_layers'], learning_rate=p['learning_rate'])
        es = callbacks.EarlyStopping(monitor='val_loss', patience=p['patience'], min_delta=p['tolerancia'], restore_best_weights=True)

        model.fit(X_train, y_train, epochs=p['epochs'], batch_size=p['batch_size'], validation_split=0.2, callbacks=[es], verbose=0)
        y_pred = model.predict(X_test)
        y_true = np.argmax(y_test, axis=1)
        y_pred_class = np.argmax(y_pred, axis=1)

        f1s = f1_score(y_true, y_pred_class, average=None)
        media = np.mean(f1s)
        std = np.std(f1s)

        resultado = {
            'parametro': param,
            'valor': val,
            'f1_media': media,
            'f1_std': std
        }
        for i, f1c in enumerate(f1s):
            resultado[f'f1_clase_{i}'] = f1c

        resultados.append(resultado)

# Exportar a Excel
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("/content/resultados_xception_variables.xlsx", index=False)
print("Resultados exportados a Excel correctamente.")


Mounted at /content/drive
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 26s 12s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step


1/2 ━━━━━━━━━━━━━━━━━━━━ 2s 3s/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
Resultados exportados a Excel correctamente.
